# Planning Larger Analysis

To answer more complex queries, BRAD can chain together multiple modules to answer a users prompt.

In [1]:
from BRAD import brad
from BRAD import llms

## Load RAG Database

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb

In [3]:
# Load the database
persist_directory = '/nfs/turbo/umms-indikar/shared/projects/RAG/databases/DigitalLibrary-10-June-2024/'
embeddings_model = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
db_name = "DigitalLibrary"
_client_settings = chromadb.PersistentClient(path=(persist_directory + db_name))
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings_model, client=_client_settings, collection_name=db_name)

/home/jpic/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/jpic/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Example Prompts

## Executing large plans

In [ ]:
brad.chat(ragvectordb = vectordb,
          llm=llms.load_openai(),
          config='/home/jpic/RAG-DEV/tutorials/planner/config-planner.json'
         )

# Scratch



## langchain

In [94]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `title` and `abstract` keys in response to the following: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

input = """User query: What is the role of PCNA in the cell cycle?

Chatlot:
...
"""

xx = json_chain.invoke({"question": input})



2024-09-18 11:29:58,499 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [98]:
xx['title']

'Role of PCNA in the cell cycle'

In [46]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import OpenAI
import logging

# Initialize the OpenAI model (or any LLM you are using)
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

# Define the logging setup if needed
log = logging.getLogger("chat_logger")

# Define a Pydantic model for the structured output (title)
class Report(BaseModel):
    title: str = Field(description="The title of the report generated from the chat log")

# Set up the parser for the output using the Pydantic model
parser = PydanticOutputParser(pydantic_object=Report)

# Define the prompt template with format instructions injected
template = """Given the following chatlog, generate a descriptive title for a report summarizing the work done. 
Please keep the title concise. This report is in response to the following user request:
{userinput}

**Chatlog**
{log}
"""

# Format the template with the user input
formatted_template = template.format(
    userinput="Write a report about PCNA in the cell cycle and cell cycle imaging",  # assuming chatstatus is already defined
    log="..."
)

# Create the PromptTemplate object
prompt = PromptTemplate(template=formatted_template, input_variables=["chatlog"])

# Combine the prompt and model into a chain
chain = prompt | llm

# Log the process
# log.debug('Calling getReportTitle', chatstatus=chatstatus)

# Get the response from the model
response = llm.invoke(formatted_template)  # assuming chathistory is already defined

# Parse the response using the Pydantic parser
parsed_report = parser.parse(response)

# Log the response for debugging
# log.debug(parsed_report, chatstatus=chatstatus)

# Access the report title
report_title = parsed_report.title


2024-09-18 10:56:32,392 - INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


OutputParserException: Failed to parse Report from completion "Understanding PCNA's Role in Cell Cycle and Advancements in Cell Cycle Imaging: A Comprehensive Report". Got: 1 validation error for Report
__root__
  Report expected dict not str (type=type_error)

In [68]:
template = """Answer the below query.\n{{format_instructions}}\n

Given the following chatlog, generate a descriptive title for a report summarizing the work done. 
Please keep the title concise. This report is in response to the following user request:
{{query}}

**Chatlog**
{log}
"""

# Format the template with the user input
formatted_template = template.format(
    log="..."
)
formatted_template

'Answer the below query.\n{format_instructions}\n\n\nGiven the following chatlog, generate a descriptive title for a report summarizing the work done. \nPlease keep the title concise. This report is in response to the following user request:\n{query}\n\n**Chatlog**\n...\n'

In [80]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

class reportPreamble(BaseModel):
	title: str = Field(description="Title of a report")
	abstract: str = Field(description="An abstract for the report")

parser = PydanticOutputParser(pydantic_object=reportPreamble)

prompt = PromptTemplate(
	template = formatted_template,
	input_variables=["query"],
	partial_variables={
		"format_instructions": parser.get_format_instructions()
	}
)
print(prompt)
chain = prompt | llm

output = chain.invoke({"query": "Write a reprot about PCNA in the cell cycle."})

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "description": "Title of a report", "type": "string"}, "abstract": {"title": "Abstract", "description": "An abstract for the report", "type": "string"}}, "required": ["title", "abstract"]}\n```'} template='Answer the below query.\n{format_instructions}\n\n\nGiven the following chatlog, generate a descriptive title for a report summarizing the work done. \nPlease keep the title concise. This report is in response to the follow

2024-09-18 11:24:19,057 - INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


In [70]:
print(prompt)

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "description": "Title of a report", "type": "string"}, "abstract": {"title": "Abstract", "description": "An abstract for the report", "type": "string"}}, "required": ["title", "abstract"]}\n```'} template='Answer the below query.\n{format_instructions}\n\n\nGiven the following chatlog, generate a descriptive title for a report summarizing the work done. \nPlease keep the title concise. This report is in response to the follow

In [62]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"title": {"title": "Title", "description": "Title of a report", "type": "string"}, "abstract": {"title": "Abstract", "description": "An abstract for the report", "type": "string"}}, "required": ["title", "abstract"]}
```


In [71]:
output

'\n"Report on PCNA in the Cell Cycle"'

In [73]:
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x14d9c12cea90>, async_client=<openai.resources.completions.AsyncCompletions object at 0x14d9c1ed26d0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [72]:
parser.invoke(output)

OutputParserException: Failed to parse reportPreamble from completion "Report on PCNA in the Cell Cycle". Got: 1 validation error for reportPreamble
__root__
  reportPreamble expected dict not str (type=type_error)

In [ ]:
from langchain.output_parsers import PydanticOutputParser

In [84]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = llms.load_openai() # OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
print(prompt_and_model.first)
output = prompt_and_model.invoke({"query": "Tell me a joke."})
print(output.content)
response = parser.invoke(output)
print(response)


input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


2024-09-18 11:25:38,651 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "setup": "Why couldn't the bicycle stand up by itself?",
  "punchline": "Because it was two tired!"
}
setup="Why couldn't the bicycle stand up by itself?" punchline='Because it was two tired!'


In [85]:
chain = prompt | model | parser
chain.invoke({"query": "Tell me a joke."})



2024-09-18 11:25:41,837 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Joke(setup="Why couldn't the bicycle stand up by itself?", punchline='Because it was two tired!')

In [92]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `title` and `abstract` keys in response to the following: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

xx = json_chain.invoke({"question": "Write a report about PCNA"})



2024-09-18 11:28:38,222 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [93]:
xx

{'title': 'Report on PCNA',
 'abstract': 'This report provides an overview of PCNA (Proliferating Cell Nuclear Antigen), a protein that plays a crucial role in DNA replication and repair processes. The report discusses the structure, function, and significance of PCNA in maintaining genomic stability and preventing mutations. Additionally, the report explores the current research and advancements in understanding the role of PCNA in cancer development and potential therapeutic strategies targeting this protein.'}

In [37]:
class Report1(BaseModel):
    title: str = Field(description="write a title for this report")
    abstract: str = Field(description="write an abstract for this report")

In [36]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Report1)

prompt = PromptTemplate(
    template="",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
response = parser.invoke(output)

'Because it was two tired!'

## guidance

In [18]:
!pip install guidance


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 6.3 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [28]:
from guidance import models, gen, select

In [25]:
model = models.OpenAI(model='gpt-3.5-turbo-0125')

In [29]:
# append text or generations to the model
model + f'Do you want a joke or a poem? A ' + select(['joke', 'poem'])


ValueError: The OpenAI model gpt-3.5-turbo-0125 is a Chat-based model and requires role tags in the prompt!             Make sure you are using guidance context managers like `with system():`, `with user():` and `with assistant():`             to appropriately format your guidance program for this type of model.

In [23]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x14d9be160dd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x14dadc6b23d0>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

## jsonformer

In [11]:
!pip install jsonformer


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [12]:
from jsonformer import Jsonformer


In [15]:
llm = llms.load_openai()

In [17]:
json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
        "courses": {
            "type": "array",
            "items": {"type": "string"}
        }
    }
}

prompt = "Generate a person's information based on the following schema:"
jsonformer = Jsonformer(llm, llm, json_schema, prompt)


/home/jpic/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


TypeError: Completions.create() got an unexpected keyword argument 'return_tensors'

In [14]:
?Jsonformer

Init signature:
Jsonformer(
    model: transformers.modeling_utils.PreTrainedModel,
    tokenizer: transformers.tokenization_utils.PreTrainedTokenizer,
    json_schema: Dict[str, Any],
    prompt: str,
    *,
    debug: bool = False,
    max_array_length: int = 10,
    max_number_tokens: int = 6,
    temperature: float = 1.0,
    max_string_token_length: int = 10,
)
Docstring:      <no docstring>
File:           ~/.local/lib/python3.11/site-packages/jsonformer/main.py
Type:           type
Subclasses:     